### Imports

In [ ]:
import os, glob, json, numpy as np, matplotlib.pyplot as plt, pandas as pd, \
    matplotlib.gridspec as gridspec, scipy.ndimage.morphology as morphology, \
    simplification.cutil as simpl, matplotlib.colors as mplcol, matplotlib.ticker as ticker, \
    warnings, regex as re, miniball, copy
from tqdm import tqdm_notebook as tqdm

from shapely.geometry import Point, LineString
from matplotlib.path import Path
import shapely.affinity

warnings.filterwarnings('ignore')

### Compute Polar Position Data

In [ ]:
# Find filenames to plot
fnames = glob.glob('Z:\\behavior\\*\\croprot\\*dlc_position_orientation.npy')

In [ ]:
# Exclude recordings from new rigs
fnames = [x for x in fnames if not ('RIG1' in x or 'RIG2' in x)]
fnames[0:3], len(fnames)

In [ ]:
# Find recording information
fnamesRecordingInfo = [os.path.join(os.path.dirname(os.path.dirname(x)), 'recording.json') for x in fnames]

def loadJSON(x):
    if os.path.exists(x):
        with open(x, 'r') as f:
            return json.load(f)
    else:
        return None
    
recordingInfo = [loadJSON(x) for x in fnamesRecordingInfo]

# Exclude recordings that are incomplete or invalid
useRecording = [('stages' in ri and (isinstance(ri['stages']['radii'], list) or ri['stages']['radii'] > 0) and \
                 ri['web_complete'] and ri['tracking_successful']) for ri in recordingInfo]

fnames = [x for x, b in zip(fnames, useRecording) if b]
recordingInfo = [x for x, b in zip(recordingInfo, useRecording) if b]

In [ ]:
fnamesShort = [re.search('^Z:/.*/(.*)/.*/.*$', fn.replace('\\', '/')).group(1) for fn in fnames]

In [ ]:
# Fill in missing stage information, if necessary
for i in range(len(recordingInfo)):
    s = recordingInfo[i]
    s['fname'] = fnames[i]
    
    # Does this recording.json file specify stage ranges, or starting points?
    if isinstance(s['stages']['protoweb'], list):
        for st in s['stages']:
            if len(s['stages'][st]) == 0:
                s['stages'][st] = []
            elif not isinstance(s['stages'][st][0], list):
                s['stages'][st] = [s['stages'][st], ]
    else:
        # Add the end of the recording
        a = np.load(s['fname'], mmap_mode='r')
        s['stages']['end'] = a.shape[0]

        if 'stabilimentum' in s['stages']:
            if s['stages']['stabilimentum'] >= 0:
                pass
            else:
                s['stages']['stabilimentum'] = s['stages']['end']
        else:
            s['stages']['stabilimentum'] = s['stages']['end']
        
        # Now convert to ranges
        s['stages']['protoweb'] = [[s['stages']['protoweb'], s['stages']['radii']],]
        s['stages']['radii'] = [[s['stages']['radii'], s['stages']['spiral_aux']],]
        s['stages']['spiral_aux'] = [[s['stages']['spiral_aux'], s['stages']['spiral_cap']],]
        s['stages']['spiral_cap'] = [[s['stages']['spiral_cap'], s['stages']['stabilimentum']],]
        s['stages']['stabilimentum'] = [[s['stages']['stabilimentum'], s['stages']['end']],]
        del s['stages']['end']
    # Convert to indices used in analysis
    arrIdx = np.load(fnames[i].replace('_position_orientation.npy','_abs_filt_interp_mvmt_noborder.idx.npy'))
    arrIdx[:] = True # TEST
    for st in s['stages']:
        for k in range(len(s['stages'][st])):
            for m in range(2):
                s['stages'][st][k][m] = np.argmin(np.abs(np.argwhere(arrIdx).T[0] - s['stages'][st][k][m]))

In [ ]:
def computePolar(fnameIdx, noPolar = False):
    # Load original data
    arr = np.load(fnames[fnameIdx])
    # Subset by index
    arrIdx = np.load(fnames[fnameIdx].replace('_position_orientation.npy','_abs_filt_interp_mvmt_noborder.idx.npy'))
    #arr = arr[arrIdx,:]
    arr[~arrIdx] = np.nan
    arr = pd.DataFrame(arr).fillna(method='ffill').fillna(method='bfill').values
    arrIdx[:] = True # TEST
    # Option to not compute polar info
    if noPolar:
        return arr
    # Determine center
    if 'center' in recordingInfo[fnameIdx]:
        center = np.array(recordingInfo[fnameIdx]['center'])
    else:
        raise Exception('Center of web should be manually specified')
    # Convert position to polar coordinates relative to approximate center
    r = np.linalg.norm(arr[:,0:2] - center[np.newaxis,:], axis=1)
    a = np.arctan2(arr[:,0] - center[np.newaxis,0], arr[:,1] - center[np.newaxis,1])
    arrPolar = np.hstack((r[:,np.newaxis], a[:,np.newaxis], arr[:,2,np.newaxis]))
    # Remove noise
    isNoise = np.linalg.norm(arr[:,0:2] - np.roll(arr, -1, axis=0)[:,0:2], axis=1) > 50
    arr[isNoise,:] = np.nan
    arrPolar[isNoise,:] = np.nan
    # Compute velocities
    arrPolarVel = np.roll(arrPolar, -25, axis=0) - arrPolar
    # Wrap rotations into -pi / +pi
    for k in [1,2]:
        for i in range(3):
            arrPolarVel[arrPolarVel[:, k] < -np.pi, k] += 2 * np.pi
        for i in range(3):
            arrPolarVel[arrPolarVel[:, k] >  np.pi, k] -= 2 * np.pi
    # Filter out
    isNoise |= (np.abs(arrPolarVel[:,0]) > 300) & (np.abs(arrPolarVel[:,1]) > np.pi)
    #
    arr[isNoise,:] = np.nan
    arrPolar[isNoise,:] = np.nan
    arrPolarVel = np.roll(arrPolar, -25, axis=0) - arrPolar
    # Wrap rotations into -pi / +pi
    for k in [1,2]:
        for i in range(3):
            arrPolarVel[arrPolarVel[:, k] < -np.pi, k] += 2 * np.pi
        for i in range(3):
            arrPolarVel[arrPolarVel[:, k] >  np.pi, k] -= 2 * np.pi
    # Compute arclength data
    arrPolarVelArclen = arrPolarVel.copy()
    arrPolarVelArclen[:,1] *= arrPolar[:,0]
    # Done!
    return arr, arrPolar, arrPolarVel, arrPolarVelArclen, center, arrIdx

### Helper function

In [ ]:
# Source: https://stackoverflow.com/questions/37765197/darken-or-lighten-a-color-in-matplotlib
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return np.clip(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]), 0, 1)

In [ ]:
def loadAVIindex(fnameIdx, idxCollapsed, suffix = 'avimapping2.npy', overwrite=True):
    # Open mapping file
    arr, arrPolar, arrPolarVel, arrPolarVelArclen, center, arrIdx = computePolar(fnameIdx)
    fnameAVImapping = glob.glob(os.path.abspath(os.path.join(os.path.dirname(
        fnames[fnameIdx]), '../raw/*_{}'.format(suffix))))[0]
    aviMapping = np.load(fnameAVImapping)
    # Already exists?
    fnameAVIidx = fnameAVImapping.replace('.npy', '') + '_idx.npy'
    if os.path.exists(fnameAVIidx) and not overwrite:
        return np.load(fnameAVIidx), aviMapping, None
    else:
        # Determine AVI chunk sizes
        import imageio
        lensAVI = []
        aviIDs = np.sort([int(re.search('-([0-9]*)\\.avi', x).group(1)) for x in glob.glob(
            fnameAVImapping.replace('_{}'.format(suffix), '*.avi'))])
        i0 = aviIDs[aviMapping[:,1].min()]
        i1 = aviIDs[aviMapping[:,1].max()]
        print('Using movie chunks {} -> {}'.format(i0, i1))
        for aviID in range(i0, i1+1):
            fnameAVI = fnameAVImapping.replace('_{}'.format(suffix), '-{:04}.avi').format(aviID)
            if not os.path.exists(fnameAVI):
                fnameAVI = fnameAVImapping.replace('_{}'.format(suffix), '-{}.avi').format(aviID)
            if not os.path.exists(fnameAVI):
                lensAVI.append(lensAVI[-1])
            else:
                r = imageio.get_reader(fnameAVI)
                lensAVI.append(len(r))
                r.close()
                del r

        # Map recording indices to AVI indices
        _aviMappingIdxs = [list(range(i0, i1+1)).index(aviIDs[x]) for x in aviMapping[:,1]]
        idxAVI = np.hstack([np.full(L, i in _aviMappingIdxs,dtype=np.bool) for L, i in zip(lensAVI, range(
            i0, i1+1))])

        _idxAVI = np.argwhere(idxAVI)[np.clip(np.argwhere(arrIdx)[idxCollapsed,0], 0, np.sum(idxAVI)-1), 0]
        print('-->', _idxAVI.shape[0], np.max(_idxAVI), np.sum(arrIdx))
        #assert(_idxAVI.shape[0] == np.sum(arrIdx))
        idxAVI[:] = False
        idxAVI[_idxAVI] = True
        # Dbg:
        _idxAVInoncollapsed = np.argwhere(idxAVI)[np.clip(np.argwhere(
            arrIdx)[:,0], 0, np.sum(idxAVI)-1), 0]
        # Save
        np.save(fnameAVIidx, idxAVI)
        return idxAVI, aviMapping, _idxAVInoncollapsed

### Set color scheme

In [ ]:
# Wong colorblind-safe palette
# Source: https://www.nature.com/articles/nmeth.1618
# Source: https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000
COLORS = [
    (0, 0, 0),
    (230, 159, 0),
#    (86, 180, 233),
    (0, 158, 115),
#    (240, 228, 66),
    (0, 114, 178),
#    (213, 94, 0),
    (204, 121, 167)
]
COLORS = [mplcol.rgb2hex(np.array(x) / 255.0) for x in COLORS]

In [ ]:
COLORS = {
    'protoweb': '#e69f00',
    'radii': '#009E73',
    'spiral_aux': '#56B4E9',
    'spiral_cap': '#CC79A7',
    'stabilimentum': '#0072B2'
}

### Plot

In [ ]:
# Source: https://stackoverflow.com/questions/2158395/flatten-an-irregular-list-of-lists
flatten = lambda *n: (e for a in n
    for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [ ]:
xy = np.array([[0, 0], [-2, 2]]).T
shape = shapely.affinity.rotate(LineString(xy).buffer(0.25), 25)
path = np.array(shape.exterior.xy).T - np.array(shape.centroid.xy).T
axisBreakMarker = Path(path)

In [ ]:
def plot(fnameIdx, fnameShort, collapsePauses, fig, spec, subpanel = 0, title=None, MAXPAUSE=600): # 10 minutes
    # Load data
    arr, arrPolar, arrPolarVel, arrPolarVelArclen, center, arrIdx = computePolar(fnameIdx)
    
    # Deep copy recording info
    recInfo = copy.deepcopy(recordingInfo[fnameIdx])

    # Collapse pauses that are longer than 30 minutes to just 30 minutes
    idxCollapsed = None
    useCache = False
    if collapsePauses:
        fnameCache = 'C:/Users/acorver/Desktop/paper-figures/tmp_collapsepauses_{}.npy'.format(
            fnamesShort[fnameIdx])
        if os.path.exists(fnameCache):
            idxCollapsed = np.load(fnameCache)
            useCache = True
        else:
            # -- Fill NaN's
            arrFl = pd.DataFrame(arr).fillna(method='ffill').fillna(method='bfill').values[:,0:2]
            # -- Compute running miniball to identify pauses
            arrFlRad = [miniball.Miniball(arrFl[max(0, i-50*MAXPAUSE//2):min(
                            arrFl.shape[0], i+50*MAXPAUSE//2),:]).squared_radius() \
                                for i in tqdm(range(0, arrFl.shape[0], 50), leave=False)]
            arrFlRad = np.repeat(arrFlRad, 50)[0:arr.shape[0]]
            # -- Identify pause segments
            idxCollapsed = np.argwhere(morphology.binary_dilation(arrFlRad > 30, iterations=50*MAXPAUSE//2))[:,0]
            np.save(fnameCache, idxCollapsed)
        # -- Subset dataset
        arr = arr[idxCollapsed,:]
        arrPolar = arrPolar[idxCollapsed,:]
        arrPolarVel = arrPolarVel[idxCollapsed,:]
        arrPolarVelArclen = arrPolarVelArclen[idxCollapsed,:]
    
    # Load stage timing info
    # -- Convert stage timing to collapsed format
    if idxCollapsed is not None:
        for st in recInfo['stages']:
            for k in range(len(recInfo['stages'][st])):
                for m in range(2):
                    recInfo['stages'][st][k][m] = np.argmin(np.abs(idxCollapsed - recInfo['stages'][st][k][m]))
    
    # Where do the even, half-hour marks occur?
    idxAVI, aviMapping, _idxAVInoncollapsed = loadAVIindex(fnameIdx, idxCollapsed, overwrite=(not useCache))
    #timestamps = np.argwhere(arrIdx)[np.argwhere(idxCollapsed),0]
    _tmp = np.full(idxCollapsed.size, np.nan, dtype=np.float32)
    _tmp[:np.sum(idxAVI)] = np.argwhere(idxAVI)[:,0]
    _tmp = pd.DataFrame(_tmp).fillna(method='ffill').values[:,0]
    timestamps = _tmp[np.argwhere(idxCollapsed)]
    # Set t=0 to start of first radii stage
    timestamps -= timestamps[recInfo['stages']['radii'][0][0], 0]
    timestampsNonCollapsed = _tmp.copy()
    tsDisplay = []
    s0 = np.ceil(np.min(timestamps) / (50 * 3600 * 0.5)) * 0.5
    for ts in np.arange(s0, np.max(timestamps) / (50 * 3600), 0.5):
        x = np.argmin(np.abs(timestamps - (50 * 3600 * ts)))
        if (len(tsDisplay) == 0 or x != tsDisplay[-1][1]):
            if (len(tsDisplay) == 0) or (x-tsDisplay[-1][1]) > np.max(timestamps)*0.0005:
                tsDisplay.append((ts, x))
    
    # PLOTTING RESOLUTION
    resVW_w = 20
    
    LIMITS = [
        [0, 5],
        [-180, 180],
        [-180, 180]
    ]

    # MEASURE THIS:
    PIXEL_TO_CM = 10.0 / 1024
    
    # Find long pauses
    pauses = np.argwhere((pd.DataFrame(timestamps[:,0]).diff() >= (50 * MAXPAUSE)).values[:,0])[:,0] / (50 * 3600)
    
    # PLOT POSITIONS
    axs = []
    for k in range(3):
        ax0 = fig.add_subplot(spec[(subpanel * 6 + k*2):(subpanel * 6 + k*2+2),0])
        
        # Set title
        if k == 0 and title is not None:
            ax0.set_title(title)
        
        # Plot background
        kmax = np.max(list(flatten([st for st in recInfo['stages'].values()])))
        for st in recInfo['stages']:
            for m in range(len(recInfo['stages'][st])):
                k0 = recInfo['stages'][st][m][0] / kmax
                k1 = recInfo['stages'][st][m][1] / kmax
                ax0.axhspan(LIMITS[k][0], LIMITS[k][1], 
                    k0, k1, facecolor = lighten_color(lighten_color(COLORS[st], 1 + 0.25 * m), 0.07))
                
                ax0.axhspan(LIMITS[k][0], LIMITS[k][0] + (LIMITS[k][1] - LIMITS[k][0]) * 0.05, 
                    k0, k1, facecolor = lighten_color(COLORS[st], 1 + 0.25 * m))

        # Simplify timeseries before plotting
        # -- Keep doubling the resVW factor until the number of points is reduced to 1% of original
        resVW = 0.000001
        while True:
            coords = np.hstack((
                np.arange(arrPolar.shape[0], dtype=np.float)[:,np.newaxis] / (3600 * 50), 
                arrPolar[:,k,np.newaxis] * PIXEL_TO_CM if k == 0 else (arrPolar[:,k,np.newaxis] * 180 / np.pi)))

            coords = coords[~np.any(np.isnan(coords), axis=1)]
            coordsSimpl = simpl.simplify_coords_vw(coords, resVW)
            if coords.shape[0] / coordsSimpl.shape[0] > 100 or coordsSimpl.shape[0] < 10000:
                coords = coordsSimpl
                break
            else:
                resVW *= 2

        ax0.plot(coords[:,0], coords[:,1], color='#444444', linewidth=1)
        
        for p in pauses:
            if p < kmax / (3600 * 50):
                ax0.scatter(p, LIMITS[k][0], marker=axisBreakMarker, s=100, 
                    color='black', clip_on=False, zorder=100)

        # Limits
        ax0.set_ylim(*LIMITS[k])
        ax0.set_xlim(0, kmax / (3600 * 50))
        
        tsDisplay = [x for x in tsDisplay if x[1] < kmax]
        ax0.set_xticks([x[1]/(50 * 3600) for x in tsDisplay])
        ax0.set_xticklabels([x[0] for x in tsDisplay], rotation=90)

        ax0.set_ylabel(['Distance from Hub\n(cm)', 'Angular Position\n(Degrees)', 'Orientation\n(Degrees)'][k])

        for brpoint in np.unique(list(flatten([st for st in recInfo['stages'].values()]))):
            ax0.axvline(brpoint / (3600 * 50), color='#111111', linestyle='--')

        if k < 2:
            ax0.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        else:
            ax0.set_xlabel('Time (Hours)')

        if k != 0:
            ax0.set_yticks([-180, -90, 0, 90, 180])
            ax0.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d°"))

        axs.append(ax0)
    
    # Plot the web
    xyAll = arr.copy()[:,0:2]

    # Load trajectory info
    xyAll = pd.DataFrame(xyAll).fillna(method='ffill').fillna(method='bfill').values.copy()

    for st in recInfo['stages']:
        sp = None
        if st == 'protoweb':
            sp = spec[(subpanel * 6 + 0):(subpanel * 6 + 3), 1]
        elif st == 'radii':
            sp = spec[(subpanel * 6 + 0):(subpanel * 6 + 3), 2]
        elif st == 'spiral_aux':
            sp = spec[(subpanel * 6 + 3):(subpanel * 6 + 6), 1]
        elif st == 'spiral_cap':
            sp = spec[(subpanel * 6 + 3):(subpanel * 6 + 6), 2]
        else:
            sp = spec[(subpanel * 6 + 3):(subpanel * 6 + 6), 2]
        ax1 = fig.add_subplot(sp)
        
        for m in range(len(recInfo['stages'][st])):
            k0 = recInfo['stages'][st][m][0]
            k1 = recInfo['stages'][st][m][1]

            xy = xyAll[k0:k1].copy()
            #xy = xy[np.linalg.norm(xy - np.array(recInfo['center']), axis=1) < 580,:]
            xy = simpl.simplify_coords_vw(xy, resVW_w)
            segmentIDs = np.cumsum(np.linalg.norm(xy - np.roll(xy, 1, axis=0), axis=1) > 200)

            for sid in np.unique(segmentIDs):
                ax1.plot(xy[segmentIDs==sid, 0], xy[segmentIDs==sid, 1], 
                         color=lighten_color(COLORS[st], 1 + 0.25 * m), linewidth=1 if st != 'stabilimentum' else 5)

        # Resize web to smallest square
        xyMin = np.ones(2) * 100 # np.min(xyAll, axis=0)
        xyMax = np.ones(2) * 924 # np.max(xyAll, axis=0)
        xyWidth  = xyMax[0] - xyMin[0]
        xyHeight = xyMax[1] - xyMin[1]
        xyCent = xyMin * 0.5 + xyMax * 0.5
        xySpan = max(xyWidth, xyHeight) / 2
        ax1.set_xlim(xyCent[0] - xySpan, xyCent[0] + xySpan)
        ax1.set_ylim(xyCent[1] - xySpan, xyCent[1] + xySpan)
        ax1.set_axis_off()
    return tsDisplay, arrIdx, idxCollapsed, timestamps, timestampsNonCollapsed, _idxAVInoncollapsed, idxAVI, aviMapping

In [ ]:
collapsePauses = True
MAXPAUSE = 60

In [ ]:
# CREATE FIGURE AND LAYOUT
fnameIdx = 7

fig = plt.figure(figsize=(24, 6))
spec = gridspec.GridSpec(ncols=3, nrows=6, width_ratios=[2.5, 0.5, 0.5])

# Draw
tsDisplay, arrIdx, idxCollapsed, timestamps, timestampsNonCollapsed, \
    _idxAVInoncollapsed, idxAVI, aviMapping = \
        plot(fnameIdx, fnamesShort[fnameIdx], True, fig, spec, MAXPAUSE=60)

# Show figure
fig.tight_layout()

# Save figure
fig.savefig('C:/Users/acorver/Desktop/paper-figures/fig_polartraj_{}{}.pdf'.format(
    fnamesShort[fnameIdx], '_limitedpauses' if collapsePauses else ''))

In [ ]:
# CREATE FIGURE AND LAYOUT
fnameIdx = 7

fig = plt.figure(figsize=(24, 6))
spec = gridspec.GridSpec(ncols=3, nrows=6, width_ratios=[2.5, 0.5, 0.5])

# Draw
tsDisplay, arrIdx, idxCollapsed, timestamps, timestampsNonCollapsed, \
    _idxAVInoncollapsed, idxAVI, aviMapping = \
    plot(fnameIdx, fnamesShort[fnameIdx], True, fig, spec, MAXPAUSE=60)

# Show figure
fig.tight_layout()

# Save figure
fig.savefig('C:/Users/acorver/Desktop/paper-figures/fig_polartraj_{}{}.pdf'.format(
    fnamesShort[fnameIdx], '_limitedpauses' if collapsePauses else ''))

In [ ]:
tasks = list(enumerate(fnamesShort))
tasks = sorted(tasks, key=lambda x: -2 if '4-5-19-a' in x[1] else (-1 if '4-22-19-a' in x[1] or '6-3-19-e' in x[1] or '4-13-19-a' in x[1] else 0))

for fnameIdx, fnameShort in tqdm(tasks):
    for collapsePauses in [True, ]:
        try:
            # CREATE FIGURE AND LAYOUT
            fig = plt.figure(figsize=(24, 6))
            spec = gridspec.GridSpec(ncols=3, nrows=6, width_ratios=[2.5, 0.5, 0.5])

            # Draw
            plot(fnameIdx, fnameShort, collapsePauses, fig, spec, MAXPAUSE=60)

            # Show figure
            fig.tight_layout()

            # Save figure
            fig.savefig('C:/Users/acorver/Desktop/paper-figures/fig_polartraj_v2_{}{}.pdf'.format(
                fnameShort, '_limitedpauses' if collapsePauses else ''))
        except Exception as e:
            print(fnameIdx, fnameShort, str(e))

### Supplementary Fig. 1

In [ ]:
fnameIdxSS = [1, 3, 13]

In [ ]:
# CREATE FIGURE AND LAYOUT
fig = plt.figure(figsize=(24, 6 * len(fnameIdxSS)))
spec = gridspec.GridSpec(ncols=3, nrows=6 * len(fnameIdxSS) , width_ratios=[2.5, 0.5, 0.5])

# Draw
for i, fnameIdx in tqdm(enumerate(fnameIdxSS), leave=False):
    plot(fnameIdx, fnamesShort[fnameIdx], True, fig, spec, subpanel=i, title=fnamesShort[fnameIdx])

# Show figure
fig.tight_layout()

# Save figure
fig.savefig('C:/Users/acorver/Desktop/paper-figures/figsupp1.pdf')

### Supplementary Fig. 1-pt2 / 2

In [ ]:
# Plot overview of remaining recordings, without polar info

In [ ]:
fnameIdxWebOnly = [i for i in range(len(fnames)) if i not in fnameIdxSS]
fnameIdxWebOnly

In [ ]:
resVW_w = 20
resVW = 20

ncols = 6
nrows = int(np.ceil(len(fnameIdxWebOnly) / ncols))

# Create figure
fig, ax = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5 * nrows))

for k, fnameIdx in tqdm(enumerate(fnameIdxWebOnly)):
    recInfo = recordingInfo[fnameIdx]
    
    ax1 = ax.flatten()[k]
    ax1.set_title(fnamesShort[fnameIdx])
    
    xyAll = pd.DataFrame(computePolar(fnameIdx, noPolar = True)).fillna(method='ffill').fillna(method='bfill').values[:,0:2]

    for st in recInfo['stages']:        
        for m in range(len(recInfo['stages'][st])):
            k0 = recInfo['stages'][st][m][0]
            k1 = recInfo['stages'][st][m][1]

            xy = xyAll[k0:k1].copy()
            #xy = xy[np.linalg.norm(xy - np.array(recInfo['center']), axis=1) < 580,:]
            xy = simpl.simplify_coords_vw(xy, resVW_w)
            segmentIDs = np.cumsum(np.linalg.norm(xy - np.roll(xy, 1, axis=0), axis=1) > 200)

            for sid in np.unique(segmentIDs):
                ax1.plot(xy[segmentIDs==sid, 0], xy[segmentIDs==sid, 1], 
                         color=lighten_color(COLORS[st], 1 + 0.25 * m), linewidth=1)

        # Resize web to smallest square
        xyMin = np.min(xyAll, axis=0)
        xyMax = np.max(xyAll, axis=0)
        xyWidth  = xyMax[0] - xyMin[0]
        xyHeight = xyMax[1] - xyMin[1]
        xyCent = xyMin * 0.5 + xyMax * 0.5
        xySpan = max(xyWidth, xyHeight) / 2
        ax1.set_xlim(xyCent[0] - xySpan, xyCent[0] + xySpan)
        ax1.set_ylim(xyCent[1] - xySpan, xyCent[1] + xySpan)
        ax1.set_axis_off()

for k in range(len(fnameIdxWebOnly), ncols * nrows):
    fig.delaxes(ax.flatten()[k])

plt.tight_layout()
plt.savefig('C:/Users/acorver/Desktop/paper-figures/suppl1-pt2.pdf'.format(resVW))